In [1]:
%%capture
%run ./load_concatenated_text.ipynb

In [ ]:
import spacy
from spellchecker import SpellChecker

nlp = spacy.load('en_core_web_sm')

spell = SpellChecker()

#Here's a list of words that aren't in the spell checker but that we should probably preserve
#We'll probably need to iterate on this list
spell.word_frequency.load_words([
    'Arlington'
    , 'Glebe'
    , 'Ballston'
    , 'Rosslyn'
    , 'Pershing'
    , 'Rockville'
    , 'MD'
    , 'VA'
    , 'Maryland'
    , 'Virginia'
    , 'Bluemont'
    , 'Wilson'
    ])

In [32]:
# Function to spell-check text. We'll use apply() to run this function on every record
def spell_check(text):
    doc = nlp(text)  # Process the text with spaCy
    corrected_words = []
    
    # Find misspelled words
    misspelled = spell.unknown([token.text for token in doc if not token.is_punct and not token.is_stop])

    for token in doc:
        if not token.is_punct and not token.is_stop and len(token.text.strip()) > 0:  # Exclude punctuation and stop words
            word = token.text.strip()
            if word.lower() in misspelled:
                correction = spell.correction(word)
                if (correction is not None) and (correction.lower() != word.lower()):
                    corrected_words.append(correction)
                    #Uncomment this line to review the list of words that are correcting
                    #print(f"Correcting {word} => {correction}")
                else:
                   corrected_words.append(word.lower())
            else:
                corrected_words.append(word)  # Preserve correct words

    if len(corrected_words)>0:
        return " ".join(corrected_words)
    else: 
        return ""





In [33]:
# Apply the spell-checking function to the DataFrame
df_all_postcards['spell_checked_text'] = df_all_postcards['concatenated_text'].apply(spell_check)

In [ ]:
#Take a look at the result
df_all_postcards